## Oracle Cloud Instance POC


In [ ]:
import os
import json
from pathlib import Path
from typing import Dict
from dataclasses import dataclass
from copy import deepcopy

import bravado
from bravado.client import SwaggerClient, CallableOperation
# from bravado.requests_client import RequestsClient
from gc3_query.lib.gc3_bravado.requests_client import OCRequestsClient
from bravado.requests_client import RequestsResponseAdapter
from bravado.swagger_model import load_file
from bravado_core.exception import MatchingResponseNotFound
from bravado.exception import HTTPBadRequest
from bravado.http_future import HttpFuture

from tinydb import TinyDB, Query

# from prettyprinter import pprint, pformat
from pprint import pprint, pformat

## https://medium.com/@betz.mark/validate-json-models-with-swagger-and-bravado-5fad6b21a825
# Validate json models with swagger and bravado
from bravado_core.spec import Spec

In [ ]:
### B01
import toml
from pathlib import Path

from gc3_query.lib import *
from gc3_query.lib import BASE_DIR
from gc3_query.lib import List, Optional, Any, Callable, Dict, Tuple, Union, Set, Generator, Path

from gc3_query.lib.gc3_config import GC3Config, IDMCredential
from gc3_query.lib.iaas_classic.iaas_requests_http_client import IaaSRequestsHTTPClient
from gc3_query.lib.iaas_classic import IaaSServiceBase, API_SPECS_DIR, IaaSRequestsHTTPClient
from gc3_query.lib.iaas_classic.instances import Instances



In [ ]:
### B01
ORACLE_VPN_CONNECTED=False
ASDF='ASDF'

In [ ]:
### B01
service = 'Instances'
idm_domain = 'gc30003'
gc3_config = GC3Config()
service_cfg = gc3_config.iaas_classic.services[service]
idm_cfg = gc3_config.idm.domains[idm_domain]
http_client: IaaSRequestsHTTPClient = IaaSRequestsHTTPClient(idm_cfg=idm_cfg)

In [ ]:
# from secrets import opc_username, opc_password
import keyring

opc_username = "eric.harris@oracle.com"
opc_password = keyring.get_password("gc3", "gc30003")
print(f"opc_username={opc_username}, opc_password={opc_password}")

In [ ]:
### B01
opc_password = keyring.get_password(service_name="gc3@gc30003", username="eric.harris@oracle.com")
print(f"opc_username={opc_username}, opc_password={opc_password}")

In [ ]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')


### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)


In [ ]:
json_data = {"user": username, "password": opc_password}
print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')

In [ ]:
proxies = {
  'http': 'http://www-proxy-ash7.us.oracle.com:80',
  'https': 'https://www-proxy-ash7.us.oracle.com:80',
}

#### Create DB to store Instance data

In [ ]:
instances_db = TinyDB('instances.tdb.json')

####  Authenticate and get header token


In [ ]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+directory+json'),
                ])
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)


# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
requests_client.session.headers.update(headers)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")

response = requests_client.session.post(url=iaas_auth_endpoint, json=json_data)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
#     print(f"\nSession Cookies: {requests_client.session.cookies}")
#     print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")
else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
# print(f"cookie_header: {cookie_header}\n")
requests_client.session.headers.update(cookie_header)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")


In [ ]:
## Update the swagger spec to use https
spec_file_path = Path().joinpath('open_api_definitions/iaas_instances.json').resolve()
spec_dict = load_file(spec_file_path)
spec_dict['schemes'] = ['https']


In [ ]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, application/oracle-compute-v3+directory+json, json, text/html')
                ])

# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                 ('Accept', 'application/oracle-compute-v3+json, json, text/html')])

In [ ]:
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)
# requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         config={'also_return_response': True,
                                                 'validate_responses': True,
                                                 'validate_requests': True,
                                                 'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
#                                          config={'also_return_response': True,
#                                                  'validate_responses': False,
#                                                  'validate_requests': False,
#                                                  'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [ ]:
for operation_name in dir(instances_resource):
    callable_operation = getattr(instances_resource, operation_name)
    print(f"{callable_operation.operation.operation_id}\t\t http_method={callable_operation.operation.http_method}\tpath={callable_operation.operation.path_name}")
    
    

In [ ]:
callable_operation

In [ ]:
@dataclass
class GetOpReturn:
    operation_details: Dict[str, Any]
    operation_response: RequestsResponseAdapter


def get_op(operation_id: str, **wkargs) -> Tuple[Dict[str, Any], RequestsResponseAdapter]:
    print(f"Operation {operation_id} starting.")
    try:
        operation_future = getattr(instances_resource, operation_id)(**wkargs)
        url = operation_future.future.request.url
        print(f"REST url for {operation_id}: {url}")
        operation_result, operation_response = operation_future.result()
    except HTTPBadRequest:
        print("Request failed for {operation_id}! ")
        print(f"URL: {operation_future.future.request.url}")
        raise
    operation_details = json.loads(operation_result)
    return GetOpReturn(operation_details,operation_response )
#     print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
#     print(f"Operation {operation_id} finished.\n")


In [ ]:
operation_id = "discoverRootInstance"
print(f"Operation {operation_id} starting.")
operation_future = getattr(instances_resource, operation_id)()
url = operation_future.future.request.url
print(f"REST url for {operation_id}: {url}")

In [ ]:
operation_result, operation_response = operation_future.result()


In [ ]:
operation_details = json.loads(operation_result)
operation_result
operation_response

In [ ]:
operation_future.future.session.headers['Accept']

In [ ]:
operation_id = "discoverRootInstance"
print(f"Operation {operation_id} starting.")
try:
    operation_future = getattr(instances_resource, operation_id)()
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

In [ ]:
### B01

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
http_future = instances.bravado_service_operations.discoverRootInstance()
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code





In [ ]:
### B01
service_response

In [ ]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")









In [ ]:
## discoverInstance
## container="Compute-587626604/"
operation_id = "discoverInstance"

print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
    container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/"
    container = "Compute-587626604/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

In [ ]:
## discoverInstance
## container="Compute-587626604/eric.harris@oracle.com/"

operation_id = "discoverInstance"
print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
#     container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/{opc_username}/"
    container="Compute-587626604/eric.harris@oracle.com/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

In [ ]:
### B01
## discoverInstance
## container="Compute-587626604/eric.harris@oracle.com/"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_container=instances.idm_user_container_name

print(f"b01_container={b01_container}")
print(f"s01_container={container}")
is_same_container = b01_container==container

print(f"is_same_container={is_same_container}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

In [ ]:
### B01

http_future = instances.bravado_service_operations.discoverInstance(container=b01_container)
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [ ]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")





In [ ]:
operation_id = "listInstance"
print(f"Operation {operation_id} starting.")
try:
    # container = operation_details['result'][0].lstrip('/').rstrip('/')
    # container = 'Compute-587626604/eric.harris@oracle.com'
    container = 'Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

In [ ]:
operation_details

In [ ]:
operation_details['result']

In [ ]:
### B01
## listInstance
## container: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_container="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"

print(f"b01_container={b01_container}")
print(f"s01_container={container}")
is_same_container = b01_container==container

print(f"is_same_container={is_same_container}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

In [ ]:
### B01

http_future = instances.bravado_service_operations.listInstance(container=b01_container)
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [ ]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")



In [ ]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)
# requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         # config={'also_return_response': True,
                                         #         'validate_responses': True,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         config={'also_return_response': True,
                                                 'validate_responses': False,
                                                 'validate_requests': False,
                                                 'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [ ]:
operation_details_copy = deepcopy(operation_details)
print(operation_details_copy)

In [ ]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details_copy['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

In [ ]:
operation_details

In [ ]:
### B01
## getInstance
## name: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_name="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

print(f"b01_name={b01_name}")
print(f"s01_name={name}")
is_same_name = b01_name==name

print(f"is_same_name={is_same_name}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

In [ ]:
### B01

http_future = instances.bravado_service_operations.getInstance(name=b01_name)
request_url = http_future.future.request.url
print(f"request_url={request_url}")
print(f"s01_url    ={request_url}")
url_same = url==request_url
print(f"url_same={url_same}")


In [ ]:
### B01

service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [ ]:
service_result = instances.bravado_service_operations.getInstance(name=b01_name).result()
service_result

```
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details_copy['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")
.
.
.
Operation getInstance starting.
name: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
REST url for getInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
Accept Header=[application/oracle-compute-v3+json, json, text/html], Content-Type Header=[application/oracle-compute-v3+json]

getInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7:
 {'account': '/Compute-587626604/default',
 'attributes': {'dns': {'domain': 'compute-587626604.oraclecloud.internal.',
                        'hostname': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.',
                        'nimbula_vcable-eth0': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.'},
                'network': {'nimbula_vcable-eth0': {'address': ['c6:b0:6d:06:65:04',
                                                                '10.19.0.246'],
                                                    'dhcp_options': [],
                                                   
```
   
#### Accept Header=[application/oracle-compute-v3+json, json, text/html], Content-Type Header=[application/oracle-compute-v3+json]

   

In [ ]:
instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_name="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

print(f"b01_name={b01_name}")
print(f"s01_name={name}")
is_same_name = b01_name==name

print(f"is_same_name={is_same_name}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

In [ ]:
accept_headers = http_future.future.session.headers['Accept']
content_type_headers = http_future.future.session.headers['Content-Type']
print(f"accept_headers={accept_headers}")
print(f"content_type_headers={content_type_headers}")

In [ ]:
### B01
## getInstance
## name: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_name="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

print(f"b01_name={b01_name}")
print(f"s01_name={name}")
is_same_name = b01_name==name

print(f"is_same_name={is_same_name}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

In [ ]:
### B01
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


http_future = instances.bravado_service_operations.getInstance(name=b01_name)
http_future.future.session.headers.update(headers)

request_url = http_future.future.request.url
print(f"request_url={request_url}")
print(f"s01_url    ={request_url}")
url_same = url==request_url
print(f"url_same={url_same}")


In [ ]:
accept_headers = http_future.future.session.headers['Accept']
content_type_headers = http_future.future.session.headers['Content-Type']
print(f"accept_headers={accept_headers}")
print(f"content_type_headers={content_type_headers}")

In [ ]:
### B01

service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [ ]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
# print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")


In [ ]:
instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])
http_future = instances.bravado_service_operations.getInstance(name=b01_name)
http_future.future.session.headers.update(headers)
service_result = instances.bravado_service_operations.getInstance(name=b01_name).result()
service_result

In [ ]:
http_client.session.headers['Accept']

In [ ]:
operation_details_copy = deepcopy(operation_details)
operation_details_copy

In [ ]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details_copy['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
# print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

In [ ]:
type(operation_details)
print(operation_details.keys())

#### Insert results in to DB

In [ ]:
instances_db.insert(operation_details)

In [ ]:
instances_db.all()[-1]

### Using BasicAuth

[https://porter.io/github.com/Yelp/bravado]


from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient

http_client = RequestsClient()
http_client.set_basic_auth(
    'api.yourhost.com',
    'username', 'password'
)
client = SwaggerClient.from_url(
    'http://petstore.swagger.io/v2/swagger.json',
    http_client=http_client,
)
pet = client.pet.getPetById(petId=42).response().result

In [19]:
from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient
from requests.auth import HTTPBasicAuth
import json


In [27]:
auth_url = 'https://compute.uscom-central-1.oraclecloud.com/authenticate/'
auth_host = 'compute.uscom-central-1.oraclecloud.com'

opc_username='eric.harris@oracle.com'
opc_username='/Compute-gc30003/eric.harris@oracle.com'
opc_password='V@nadium123!'
data={"user":opc_username, "password":opc_password}
spec_url = 'https://apicatalog.oraclecloud.com/webapp/v1/orgs/oracle-public/apicollections/compute/18.1.2/apis/Instances/canonical'
print(data)

{'user': '/Compute-gc30003/eric.harris@oracle.com', 'password': 'V@nadium123!'}


In [28]:

http_client = RequestsClient()
http_client.set_basic_auth(
    auth_host,
    opc_username, opc_password
)
http_client
# client = SwaggerClient.from_url(
#     spec_url,
#     http_client=http_client,
# )


<class 'bravado.requests_client.RequestsClient'>()

In [29]:
r = http_client.session.post(url=auth_url, auth=HTTPBasicAuth(opc_username, opc_password))

In [30]:
r = http_client.session.post(url=auth_url, data=data, params=data, auth=HTTPBasicAuth(opc_username, opc_password))

In [31]:
r.text

'{"message": "Unsupported Media Type"}'

In [14]:
r.cookies

<RequestsCookieJar[]>

In [6]:
http_client.session.cookies.keys()

[]

In [7]:
r.text

'{"message": {"password": "Field is required", "user": "Field is required"}}'